In [1]:
import bhnerf
from astropy import units
import jax

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd

# Runing on 2 GPUs
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3'

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-endt1obe because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Welcome to eht-imaging! v 1.2.2 



2022-12-15 14:41:16.952255: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /.singularity.d/libs


# Generate synthetic ALMA lightcurves 
---
Polarized lightcurves corresponding to ALMA observation times on April 11, 2017. \
Generate synthetic hot-spot data and the resulting polarized image-plane.

In [2]:
"""
Load ALMA Data and time average over a window
"""
data_path = '../data/LC_EHT2017_V30_I25_K15/AA_SELFC_LO_Apr11.dat'
alma_lc = pd.read_csv(data_path)

t0 = 9. + 20./60.           # UTC
t7 = t0 + 68./60. + 35./60. # UTC

alma_lc_loops = alma_lc.loc[np.bitwise_and(alma_lc['time']>=t0, alma_lc['time']<=t7)]
window_size = 8
alma_lc_means = alma_lc_loops.rolling(window_size).mean().loc[::window_size].dropna()
t_frames = alma_lc_means['time'].values * units.hr

%matplotlib widget
plt.figure(figsize=(5,3))
plt.errorbar(alma_lc.time,  alma_lc['amp'], yerr=alma_lc['sigma'], fmt='.', ecolor='black')
plt.errorbar(alma_lc_means.time,  alma_lc_means['amp'], fmt='.', color='tab:orange')
plt.fill_between([t0, t7], [3.0, 3.0], alpha=0.3, color='gray')
plt.legend()
plt.ylim([1.5, 3])
plt.title('ALMA lightcurve')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


Text(0.5, 1.0, 'ALMA lightcurve')

In [3]:
"""
Compute geodesics, azimuthal velocity field, stokes factors
"""
fov_M = 30.0 
spin = 0.2
inclination = np.deg2rad(20.0)
rmax = fov_M / 2

# Compute geodesics (see Tutorial1)
geos = bhnerf.kgeo.image_plane_geos(
    spin, inclination, 
    num_alpha=64, num_beta=64, 
    alpha_range=[-fov_M/2, fov_M/2],
    beta_range=[-fov_M/2, fov_M/2]
)
t_injection = -float(geos.r_o)
rmin = float(geos.r.min())

# Keplerian prograde velocity field
Omega = np.sign(spin + np.finfo(float).eps) * np.sqrt(geos.M) / (geos.r**(3/2) + geos.spin * np.sqrt(geos.M))
umu = bhnerf.kgeo.azimuthal_velocity_vector(geos, Omega)
g = bhnerf.kgeo.doppler_factor(geos, umu)

# To match ~hs inensity=0.3, ~hs pol=0.15 (https://www.aanda.org/articles/aa/pdf/2022/09/aa44493-22.pdf)
flux_scale = 0.05
Q_frac = 0.8
V_frac = 0.1

# Polarized emission
b_consts = [0.0, -1.0, 0.0]
b = bhnerf.kgeo.magnetic_field(geos, *b_consts) 
J = np.nan_to_num(bhnerf.kgeo.parallel_transport(geos, umu, g, b, Q_frac=Q_frac, V_frac=V_frac), 0.0)

/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/xarray/core/computation.py:700: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


In [6]:
"""
Generate synthetic hot-spot lightcurves matching ALMA observation times.
"""
rot_angle = np.deg2rad(180)
orbit_radius = 10.0
hs_std = 1.0

# Generate hotspot measurements (see Tutorial2)
emission_hs = flux_scale * bhnerf.emission.generate_hotspot_xr(
    resolution=(64, 64, 64), 
    rot_axis=[0.0, 0.0, 1.0], 
    rot_angle=rot_angle,
    orbit_radius=orbit_radius,
    std=hs_std,
    r_isco=bhnerf.constants.isco_pro(spin),
    fov=(fov_M, 'GM/c^2')
)
image_plane_hs = bhnerf.emission.image_plane_dynamics(emission_hs, geos, Omega, t_frames, t_injection, J)
lc_hs = image_plane_hs.sum(axis=(-1,-2))

/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/xarray/core/computation.py:700: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


In [7]:
%matplotlib widget
movie_list = [xr.DataArray(image_plane_hs[:,i], dims=['t','alpha','beta']) for i in range(image_plane_hs.shape[1])]
fig, axes = plt.subplots(1, 4, figsize=(10, 3))
bhnerf.visualization.animate_movies_synced(movie_list, axes, titles=['I', 'Q', 'U', 'V'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
%matplotlib widget
bhnerf.visualization.plot_stokes_lc(lc_hs, t_frames, add_mean=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Recover 3D emission 
---
Recover the unknown 3D emission directly from the polarized lightcurves using bh-NeRF. \
This recovery is an idealized recovery with no systematic noise modeling.

## No background accretion

In [7]:
import warnings
warnings.simplefilter("ignore")

seed = 1
batchsize = 6
hparams = {'num_iters': 100000, 'lr_init': 1e-4, 'lr_final': 1e-6}

lc_total = image_plane_hs.sum(axis=(-1,-2))
sigma = np.ones_like(lc_total)
sigma[:] = [1, 0.1, 0.1]

train_step = bhnerf.optimization.TrainStep.image(t_frames, lc_total, sigma, dtype='lc')
predictor = bhnerf.network.NeRF_Predictor()
raytracing_args = bhnerf.network.raytracing_args(geos, Omega, t_injection, t_frames[0], rmax, J)
params = predictor.init_params(raytracing_args, seed=seed)
state = predictor.init_state(params, **hparams)

runname = 'bg_noisy/alma_model.no_noise.spin_{:.1f}.inc_{:.1f}.rot{:.1f}.initkey{}'.format(
    spin, np.rad2deg(inclination), np.rad2deg(rot_angle), seed)

state = bhnerf.optimization.run(
    runname, batchsize, hparams['num_iters'], state, train_step,
    raytracing_args, rmin, rmax, emission_true=emission_hs)

iteration:   0%|          | 0/100001 [00:00<?, ?it/s]

## Stochastic background accretion

In [5]:
"""
Load a 2D Gaussian Random Field (GRF) and expand it to 3D.
pynoisy: https://github.com/aviadlevis/pynoisy
"""
grf = xr.load_dataarray('../data/grf.seed12.nc')
t_interp = float(grf.t[-1]) * (t_frames - t_frames.min()) / (t_frames.max() - t_frames.min())
image_plane_bg = bhnerf.emission.grf_to_image_plane(grf.interp(t=t_interp), geos, Omega, J, alpha=2.0)

# From https://www.aanda.org/articles/aa/pdf/2022/09/aa44493-22.pdf
I_sha, P_sha, V_sha = 2.4, 0.2, -0.025
image_plane_bg = bhnerf.emission.normalize_stokes(image_plane_bg, I_sha, P_sha, V_sha)
lc_bg = image_plane_bg.sum(axis=(-1,-2))

image_plane = image_plane_hs + image_plane_bg
lc_total = image_plane.sum(axis=(-1,-2))

/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/xarray/core/computation.py:700: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


In [6]:
%matplotlib widget
movie_list = [xr.DataArray(image_plane[:,i], dims=['t','alpha','beta']) for i in range(image_plane.shape[1])]
fig, axes = plt.subplots(1, 4, figsize=(10,3))
bhnerf.visualization.animate_movies_synced(movie_list, axes, titles=['I', 'Q', 'U', 'V'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
%matplotlib widget
bhnerf.visualization.plot_stokes_lc(lc_bg, t_frames)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
import warnings
warnings.simplefilter("ignore")

seed = 2
batchsize = 6
hparams = {'num_iters': 100000, 'lr_init': 1e-4, 'lr_final': 1e-6}

sigma = np.ones_like(lc_total)
sigma[:] = [1, 0.1, 0.1]

train_step = bhnerf.optimization.TrainStep.image(t_frames, lc_total, sigma, dtype='lc')
predictor = bhnerf.network.NeRF_Predictor()
raytracing_args = bhnerf.network.raytracing_args(geos, Omega, t_injection, t_frames[0], rmax, J)
params = predictor.init_params(raytracing_args, seed=seed)
state = predictor.init_state(params, **hparams)

runname = 'bg_noisy/alma_model.spin_{:.1f}.inc_{:.1f}.rot{:.1f}.stokes_norm.seed12.initkey{}'.format(
    spin, np.rad2deg(inclination), np.rad2deg(rot_angle), seed)

state = bhnerf.optimization.run(
    runname, batchsize, hparams['num_iters'], state, train_step,
    raytracing_args, rmin, rmax, emission_true=emission_hs)

iteration:   0%|          | 0/100000 [00:00<?, ?it/s]

# Visualize recovery

In [12]:
def trim_to_gpu_num(array):
    output = array
    if output.shape[0] % jax.device_count():
        output = output[:-1]
    return output

checkpoint_dir = '../checkpoints/bg_noisy/alma_model.no_noise.spin_0.2.inc_20.0.rot180.0.initkey2/'
batchsize = 20

t_frames = trim_to_gpu_num(t_frames)
target = trim_to_gpu_num(lc_hs)

train_step = bhnerf.optimization.TrainStep.image(t_frames, target, dtype='lc')
predictor = bhnerf.network.NeRF_Predictor()
raytracing_args = bhnerf.network.raytracing_args(geos, Omega, t_injection, t_frames[0], rmax, J)
params = predictor.init_params(raytracing_args)
state = predictor.init_state(params, checkpoint_dir=checkpoint_dir)

_, movie = bhnerf.optimization.total_movie_loss(
    checkpoint_dir, batchsize, state, train_step, raytracing_args, rmax, return_frames=True
)
lc_est = movie.sum(axis=(-1,-2))

/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/xarray/core/computation.py:700: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)
/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:390: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:377: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  warnings.warn(


In [15]:
%matplotlib widget
movie_list = [xr.DataArray(movie[:,i], dims=['t','alpha','beta']) for i in range(movie.shape[1])]
fig, axes = plt.subplots(1, 4, figsize=(10, 3))
bhnerf.visualization.animate_movies_synced(movie_list, axes, titles=['I', 'Q', 'U', 'V'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
%matplotlib widget
fig, axes = plt.subplots(1, 5, figsize=(12,3))
bhnerf.visualization.plot_stokes_lc(target, t_frames, axes=axes, label='True')
bhnerf.visualization.plot_stokes_lc(lc_est, t_frames, axes=axes, color='r', linestyle='--', label='Estimate')
for ax in axes:
    ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
emission_grid = bhnerf.network.sample_3d_grid(predictor.apply, state.params, float(geos.r.min()), rmax, fov=fov_M)
bhnerf.visualization.ipyvolume_3d(emission_grid, fov=fov_M, level=[0, 0.2, 0.7])